Данный ноутбук посвящён построению предсказательных моделей. Обзор данных - в ноутбуке ```data-preparation```.  Постановка задачи - в файле ```BST_Задача_Моделирование_вкладов.docx```

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline 
import matplotlib.pyplot as plt

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

- Данные для обучения: наблюдения признаков в 2011-2017г. включительно (```df```).
- Данные для проверки: наблюдения признаков за 2018г(```df_m-df```).

In [ ]:
df = pd.read_csv('../data/BST_DataSet.csv', encoding='utf-8')

In [ ]:
df_m = pd.read_csv('../data/BST_DataSet+mine1.csv', encoding='utf-8')

In [ ]:
df_m.shape

Настроим индекс по датам наблюдения

In [ ]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
df.index = df.date
del(df['date'])

In [ ]:
df_m.date = pd.to_datetime(df_m.date, format='%m/%d/%Y')
df_m.index = df_m.date
del(df_m['date'])

In [ ]:
df_m.shape

Посмотрим на данные в нормированном виде. По некоторым наблюдениям отличимы тренды, периоды (об этом позже),  заметен аномальный интервал в окрестности января 2015г.:

In [ ]:
def normalizeData(df, plot=False):
    df_n = df.copy()
    for i in df_n.columns:
        m = max(df_n[i])
        df_n[i] = df_n[i].map(lambda x: x/m)
        if plot:
            plt.plot(df_n[i],label=i)
    return df_n

In [ ]:
## plot normalized column values
plt.figure(figsize=(12,5))
df_n = normalizeData(df, plot=True)


Обратим внимание, что значения целевой переменной имеют сезонный характер. Построим ежегодные графики:

In [ ]:
## plot year-long stats
pv = pd.pivot_table(df, index=df.index.month, columns=df.index.year,values=['delta'])
pv.plot(figsize=(14,5), title='Ежегодные графики:')

Построим матрицу попарных корреляций признаков, поскольку мы имеем подозрение на рыночную взаимосвязь некоторых из них. В частности, подозрение падает на пары ```денежная масса-зарплата```, ```инфляция-зарплата``` (в нашем случае ```ИПЦ``` - показатель инфляции).

Предположим интуитивно, что при повышении объёма денежной массы (например, при эмиссии денег) деньги при прочих равных обесцениваются, товары и услуги становятся дороже в денежном выражении, а соответственно, растут и зарплаты, складывающиеся из платы за эти услуги.  

На "карте температур" также заметно влияние цены на нефть и курса доллара 

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.corr()

Построим диаграммы рассеяния признаков

In [ ]:
sns.pairplot(data=df)

Выделим те признаки, на которые у нас есть подозрение в линейной зависимости. Некоторые пары имеют около-кубическую зависимость на вид.

In [ ]:
sns.pairplot(data=df[['avg_payment', 'buyer_prices_index','m0', 'm2']])

Так как условие задачи требует от нас интерпретируемости результатов, и количество наблюдений относительно невелико, то попробуем использовать в качестве предсказательной модели линейную регрессию.

Попробуем вначале наивный подход "в лоб", взяв данные в практически неизменном виде, пропустив проверку необходимых условий для работы линейной регрессии.

### Test-train separation

In [ ]:
#test size is in months specially for our task
def prepareData(df, target_col='y',  test_index=0):
    df1 = df.copy()
    #if we have separate test-train files/datasets, use this
    if test_index==0:
        X, y = df1.drop([target_col], axis=1), df1[target_col]
        return X, y
    
    X_train = df1[:test_index].drop([target_col], axis=1)
    y_train = df1[:test_index][target_col]
    X_test = df1[test_index:].drop([target_col], axis=1)
    y_test = df1[test_index:][target_col]
    
    del(df1)
    return X_train, X_test, y_train, y_test

### Modelling: Linear

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error

Проверим нашу модель на предоставленных заказчиком данных, построив для примера прогноз в 2017г. (соответственно, последние 12 месяцев исходного датасета: ```test_index=-12```)

In [ ]:
X_train_2017, X_test_2017, y_train_2017, y_test_2017 = prepareData(df, target_col='delta', test_index=-12)

In [ ]:
def predict(X_train, X_test, y_train, y_test, plot=True):
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    prediction = lr.predict(X_test)
    if plot:
        plt.figure(figsize=(15, 7))
        plt.plot(prediction, "r", label="prediction")
        plt.plot(y_test.values, label="actual")
        plt.legend(loc="best")
        plt.title("Linear regression\n Mean absolute error {}".format(round(mean_absolute_error(prediction, y_test))))
        plt.grid(True);
    return lr, prediction

In [ ]:
predict(X_train_2017, X_test_2017, y_train_2017, y_test_2017)

Видим, что выбранная предсказательная модель вполне годится для применения в первом приближении.

Обратимся к данным, включающим 2018 год:

In [ ]:
X_train, X_test, y_train, y_test = prepareData(normalizeData(df_m), target_col='delta', test_index=-12)

In [ ]:
lr1, prediction1 = predict(X_train, X_test, y_train, y_test)

Как мы видим, при прогнозировании 2018г. на данных из открытых источников, которые "не бьются" с исходными (см. ноутбук ```data-preparation.ipynb```), получаем достаточно грубые результаты. "Garbage in - garbage out."

Посмотрим, какие параметры имеют наибольшее влияние на значение целевой переменной.

In [ ]:
plt.plot(lr1.coef_)

Попробуем изменить поведение модели, избавившись от какого-то числа несущественных (или линейно зависимых друг от друга) признаков. Для этого воспользуемся методом главных компонент.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
def reduce(df, components=5):
    pca = PCA(components)
    XPCAreduced = pca.fit_transform(df)
    df_reduced = pd.DataFrame(XPCAreduced)
    return df_reduced

In [ ]:
df_t = normalizeData(df_m)
df_t = df_t.drop(['delta'],axis=1)
df_reduced = reduce(df_t, 6)
del(df_t)

In [ ]:
#sns.pairplot(df_reduced)

In [ ]:
for i in df_reduced.columns:
    print(np.corrcoef(df_m['delta'],df_reduced[i])[0][1])

In [ ]:
df_n = normalizeData(df_m, plot=False)

In [ ]:
X_train = df_reduced[:-12]
X_test = df_reduced[-12:]
y_train = df_n['delta'][:-12]
y_test = df_n['delta'][-12:]

In [ ]:
lr2, prediction2 = predict(X_train, X_test, y_train, y_test)

Для временных рядов полезны некоторые эвристики, например добавление признаков 'значение вчера', 'значение год назад'. Добавим их вместе с номером текущего месяца, года.

In [ ]:
def addHeuristics(df, avg=False, previous_year=False, previous_month=False):
    df1 = df.copy()
    df1['month'] = df1.index.month
    df1['year'] = df1.index.year
    if previous_year:
        df1['previous_year_delta'] = df1['delta'].shift(12)
        df1 = df1[12:]
    if previous_month:
        df1['previous_month_delta'] = df1['delta'].shift(1)
        df1 = df1[1:]
    return df1

In [ ]:
df_h = addHeuristics(df_m, previous_year=True)

In [ ]:
df_h.shape

In [ ]:
X_train, X_test, y_train, y_test = prepareData(addHeuristics(normalizeData(df_m)), target_col='delta', test_index=-12)

In [ ]:
lr3, prediction3 = predict(X_train, X_test, y_train, y_test, plot=False)

Посмотрим на коэффициенты, получившиеся при работе регрессии.

In [ ]:
lr3.coef_

In [ ]:
plt.plot(lr3.coef_)

Заметим, что признаки линейной регрессии идут в том же порядке, что и колонки в датасете. Взглянем на них:

In [ ]:
addHeuristics(df_m).columns

Как мы можем видеть из результатов, важен седьмой, восьмой, десятый признак (на графике их номер на единицу меньше). Таким образом, на прирост денег на рублёвых депозитах, согласно нашей модели, влияют соответственно показатели М2, М0, размер зарплат, курс доллара и далее в порядке снижения "амплитуды".

Построим графики прогноза с учётом эвристик.

- С учётом прошлогоднего значения:

In [ ]:
X_train, X_test, y_train, y_test = prepareData(normalizeData(addHeuristics(df_m, previous_year=True)), target_col='delta', test_index=-12)

In [ ]:
lr3, prediction3 = predict(X_train, X_test, y_train, y_test)

- С учётом значения в прошлом году:

In [ ]:
X_train, X_test, y_train, y_test = prepareData(normalizeData(addHeuristics(df_m, previous_year=True, previous_month=True)), target_col='delta', test_index=-12)

In [ ]:
lr4, prediction4 = predict(X_train, X_test, y_train, y_test)

Видим, что несмотря на потерю 13 строк наблюдений из 84, добавление новых признаков положительно влияет на поведение модели.

In [ ]:
df_n = normalizeData(df_m, plot=False)

In [ ]:
df_t = addHeuristics(df_m, previous_year=True, previous_month=True)
df_t = df_t.drop(['delta'],axis=1)
df_reduced = reduce(df_t, 11)
del(df_t)

In [ ]:
X_train = df_reduced[:-12]
X_test = df_reduced[-12:]
y_train = df_n['delta'][13:-12]
y_test = df_n['delta'][-12:]

In [ ]:
lr5, prediction5 = predict(X_train, X_test, y_train, y_test)

### Conclusion

Вывод: выбранная модель линейной регрессии оказалась вполне применима в целях прогнозирования прироста/убывания объёма рублёвых депозитов. Для повышения точности результатов можно:
- повысить число наблюдений, возможно, учесть зарубежный опыт.
- добавить новые признаки, не предоставленные заказчиком: например, можно предположить, что объём вкладов зависит от числа банков, работающих с физическими лицами. Такие данные, с разбивкой числа банков по их капиталу, предоставляет ЦБ. Например, нами не было изучено, как влияет поведение целевой величины при ликвидации банков, при действии АСВ и проч. Также не были учтены демографические данные и действие внешнеполитических факторов.
- использовать более сложные, возможно, не столь понятные/интерпретируемые модели.